In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("/Users/gillestondeur/code/syeda-tabassum-rahaman/scam-job-detector/raw_data/fake_job_postings.csv")

In [4]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
df.function.nunique()

37

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [6]:
df.isnull().sum()


job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [10]:
df['fraudulent'].value_counts(normalize=True)

fraudulent
0    0.951566
1    0.048434
Name: proportion, dtype: float64

# proportion of fraudulent vs non-fraudulent job postings skwewed --> address to handle class imbalance

In [16]:
req =df.location.nunique()
req

3105

In [18]:
df.sample(100)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
8452,8453,Senior Digital Producer,"US, CA, Los Angeles",NaN,100000-110000,Searching for the perfect creative agent or mi...,Senior Digital ProducerLooking to work with on...,At least 7 to 10 years of 360 digital producin...,NaN,0,1,0,Contract,Mid-Senior level,Bachelor's Degree,Marketing and Advertising,Project Management,0
4694,4695,Accountant,"GB, WSX, Crawley",NaN,NaN,NaN,Our Client business based in Crawley is lookin...,You will be a qualified accountant from any d...,£45000 plus company car and additional benefits,0,1,0,Full-time,NaN,NaN,NaN,NaN,0
9910,9911,"Operations Supervisor - Urbana, IL","US, OH, Columbus",NaN,NaN,We Provide Full Time Permanent Positions for m...,Participating or leading in developing correct...,"Job is in Urbana, ILParticipating or leading i...",NaN,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Facilities Services,Manufacturing,0
13811,13812,UI/UX Designer for Web and Mobile Application,"US, FL, Fort Lauderdale",NaN,NaN,Hello Show is transforming the way real estate...,Hello Show is a design-driven real estate appl...,"As our UI/UX Designer, you will be at the fore...",NaN,0,1,1,Full-time,NaN,NaN,Information Technology and Services,Design,0
8259,8260,Materials Manager - SAP data analysis - Greenw...,"US, AZ, Phoenix",NaN,NaN,We Provide Full Time Permanent Positions for m...,Managing the Materials Department including pu...,Materials Manager - SAP data analysis - Greenw...,NaN,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Management Consulting,Management,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666,8667,Consulting Operations Manager,"US, OR, Portland",NaN,NaN,Can data be a thing of beauty? We think so.At ...,About SeabourneSeabourne (#URL_c706f4c48cf2091...,Required experience and skills4+ years of proj...,"This position is based in our Portland, Oregon...",0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Consulting,0
5224,5225,Graduate Program,"SG, ,",Sales,NaN,If working in a cubical seems like your idea o...,We’re looking for Graduates to join the TradeG...,Excellent Business Level English.Positive outl...,NaN,0,1,1,Full-time,Entry level,Bachelor's Degree,Information Technology and Services,Sales,0
173,174,Technician Instrument & Controls,US,Power Plant & Energy,NaN,Edison International and Refined Resources hav...,Technician Instrument &amp; ControlsLocation D...,JOB QUALIFICATIONS-Ability to understand proce...,"we are a team of almost 8,000 employees who he...",0,1,1,Full-time,Mid-Senior level,Certification,Electrical/Electronic Manufacturing,Other,1
17611,17612,ObjC Developer,"PL, MZ, Warsaw",NaN,NaN,NaN,inFullMobile sp. z o.o. is a mobile software d...,What we expect:&bull; A proven track record in...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,1


In [21]:
temp = df.has_questions.value_counts()
temp

has_questions
0    9088
1    8792
Name: count, dtype: int64